In [15]:
import json
import os
from os import listdir
import pandas
from tqdm import tqdm as tqdm
import re

In [2]:
import spacy
nlp = spacy.load('en_core_web_md')
#doc = nlp("Hello how are you !")

In [3]:
path = "Ann"
files = [ f for f in listdir(path) if f.split('.')[1] == 'csv']
print (files)

['6.csv', '7.csv', '5.csv', '4.csv', '1.csv', '3.csv', '2.csv', '14.csv', '10.csv', '11.csv', '13.csv', '12.csv', '9.csv', '8.csv']


In [4]:
allData = []
for eachFile in files:
    base = os.path.basename(eachFile).split('.')[0]
    #print (base)
    ff = os.path.join(path,eachFile)
    data = pandas.read_csv(ff,header = None,encoding="utf-8")
#     print (data)
#     input()
    data[[3]] = data[[3]].fillna('-')
    data[[4]] = data[[4]].fillna('-')
    data = data.values.tolist()
    allData.append(data)

In [5]:
doc = nlp("It can be seen")
for g in doc:
    print (g,g.tag_)
print (len(allData))

It PRP
can MD
be VB
seen VBN
14


In [6]:
# def nodeAccuracy(output):
#     ''' Calculates the accuracy of the graph nodes '''
    
#     total = 0
#     correct = 0
#     for each in output:
#         gold,pred = each[0],each[1]
#         #print (gold,pred,each)
#         if gold != '-':
#             total += 1
#             if pred != '-/-':
#                 if (pred == gold):
#                     correct += 1
#                 else:
#                     pred = pred.split('/')
#                     for i in pred:
#                         if i in gold and len(gold)==1:
#                             correct += 1
#     accuracy = round(correct / total,2)
#     print (total,correct,accuracy)


In [7]:
def nodeAccuracy(output):
    ''' Calculates the accuracy of the graph nodes '''
    
    # Penalize half if partially correct
    total = 0.0
    correct = 0.0
    for each in output:
        total += 1.0
        gold,pred = each[0],each[1]
        if gold == pred:
            correct+=1.0
        elif ((gold in pred) or (pred in gold)) and gold and pred:
            correct+=0.5
        else:
            correct+=0.0
        print (gold,pred,correct)
    accuracy = round(correct / total,2)
    print (total,correct,accuracy)
            

In [31]:
# actionRules = ['PRP-MD-VB','PRP-RB-VB','PRP-VB',
#                 'PRP-MD-VBG','PRP-RB-VBG','PRP-VBG',
#                 'PRP-MD-VBN','PRP-RB-VBN','PRP-VBN',
#                 'PRP-MD-VBP','PRP-RB-VBP','PRP-VBP',
#                'PRP-MD-RB-VB'
#                 #'PRP-MD-VBZ','PRP-RB-VBZ','PRP-VBZ',
#                 'PRP-MD-VBD','PRP-RB-VBD','PRP-VBD']
# infoRules = ['EX-VBD','EX-VBP','EX-VBZ','VBD-IN','PRP-VBZ',
#             'NN-VBD-VBN','NNS-VBD-VBN','NN-VBD-VBG','NNS-VBD-VBG',
#             'NN-VBP-VBN','NNS-VBP-VBN','NN-VBP-VBG','NNS-VBP-VBG',
#             'NN-VBZ-VBN','NNS-VBZ-VBN',
#             'NN-MD-VB','NN-MD-RB-VB']

# filterPos = 'PDT|DT|IN|,|:|.|JJ|RB|RP'
# output = []
# #print (allData[1])
# for eachFile in allData:
#     for docid,lineno,content,enttype,edges in tqdm(eachFile):
#         print (docid,lineno,content,enttype,edges)
#         #input()
#         if enttype.strip()=="-":
#             continue
#         posTag = []
#         for word in nlp(content):
#             posTag.append(word.tag_)
#         posTagSeq = "-".join(posTag)
#         # Removing Adverbs(RB%, RP%), Adjective(PDT,JJ%), Conjunction &support preposition(IN), Determiner(DT), Punctuation (,:.)
        
#         reducedPosTagSeq = [x for x in posTag if not re.search(filterPos,x) ]

#         print ("reducedPosTagSeq:",reducedPosTagSeq)
#         print (posTag,posTagSeq)
#         predEnttypeAction = ''
#         for eachRule in actionRules:
#             if eachRule in posTagSeq:
#                 predEnttypeAction = 'A'
#                 print ("Rule:",eachRule)
#                 break
#         predEnttypeInfo = ''
#         for eachRule in infoRules:
#             if eachRule in posTagSeq:
#                 predEnttypeInfo = 'I'
#                 print ("Rule:",eachRule)
#                 break

#         if enttype in ["A,I","I,A","M"]:
#             enttype = "A/I"
#         pred=""
#         if predEnttypeAction and predEnttypeInfo:
#             pred = predEnttypeAction+"/"+predEnttypeInfo
#         else:
#             pred = predEnttypeAction+predEnttypeInfo
            
#         output.append([enttype,pred])
#         print (enttype, predEnttypeAction,predEnttypeInfo,",",pred)
#         #input()
# nodeAccuracy(output)





  0%|          | 0/34 [00:00<?, ?it/s]



 35%|███▌      | 12/34 [00:00<00:00, 112.39it/s]

6 1 We are given a nodejs API which we can read/write the notes A/I -
reducedPosTagSeq: ['PRP', 'VBP', 'VBN', 'NNS', 'NN', 'PRP', 'MD', 'VB', 'SYM', 'VB', 'NNS']
['PRP', 'VBP', 'VBN', 'DT', 'NNS', 'NN', 'WDT', 'PRP', 'MD', 'VB', 'SYM', 'VB', 'DT', 'NNS'] PRP-VBP-VBN-DT-NNS-NN-WDT-PRP-MD-VB-SYM-VB-DT-NNS
Rule: PRP-MD-VB
A/I A  , A
6 2 Follow the instruction on the landing page and visit /help, and we can see a hidden file fetcher API: A -
reducedPosTagSeq: ['VB', 'NN', 'IN', 'NN', 'NN', 'CC', 'VB', 'NNP', ',', 'CC', 'PRP', 'MD', 'VB', 'VBN', 'NN', 'NN', 'NN', ':']
['VB', 'DT', 'NN', 'IN', 'DT', 'NN', 'NN', 'CC', 'VB', 'NNP', ',', 'CC', 'PRP', 'MD', 'VB', 'DT', 'VBN', 'NN', 'NN', 'NN', ':'] VB-DT-NN-IN-DT-NN-NN-CC-VB-NNP-,-CC-PRP-MD-VB-DT-VBN-NN-NN-NN-:
Rule: PRP-MD-VB
A A  , A
6 3 Okay, let's try to guess the filename of the server A -
reducedPosTagSeq: ['UH', ',', 'VB', 'PRP', 'VB', 'TO', 'VB', 'NN', 'IN', 'NN']
['UH', ',', 'VB', 'PRP', 'VB', 'TO', 'VB', 'DT', 'NN', 'IN', 'DT', 'NN'] U





 82%|████████▏ | 28/34 [00:00<00:00, 123.34it/s]



100%|██████████| 34/34 [00:00<00:00, 130.69it/s]



  0%|          | 0/17 [00:00<?, ?it/s]



 82%|████████▏ | 14/17 [00:00<00:00, 133.66it/s]



100%|██████████| 17/17 [00:00<00:00, 129.27it/s]



  0%|          | 0/31 [00:00<?, ?it/s]

reducedPosTagSeq: ['PRP', 'VBZ', 'JJ', 'PRP', 'VBD', 'RB', 'VB', 'NN', ',', 'IN', 'PRP', 'VBP', 'TO', 'VB', 'JJ', 'NN', 'IN', 'NN', 'NN', '.']
['PRP', 'VBZ', 'JJ', 'PRP', 'VBD', 'RB', 'VB', 'DT', 'NN', ',', 'IN', 'PRP', 'VBP', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN', 'NN', '.'] PRP-VBZ-JJ-PRP-VBD-RB-VB-DT-NN-,-IN-PRP-VBP-TO-VB-DT-JJ-NN-IN-DT-NN-NN-.
Rule: PRP-VB
Rule: PRP-VBZ
A/I A I , A/I
6 29 so I assume it must be /. A -
reducedPosTagSeq: ['CC', 'PRP', 'VBP', 'PRP', 'MD', 'VB', '.']
['CC', 'PRP', 'VBP', 'PRP', 'MD', 'VB', '.'] CC-PRP-VBP-PRP-MD-VB-.
Rule: PRP-MD-VB
A A  , A
6 30 Failed Attempts A -
reducedPosTagSeq: ['JJ', 'NNS']
['JJ', 'NNS'] JJ-NNS
A   , 
6 31 But the http module works well with ../, so I think it's not the vulnerable version. A/I 30,31
reducedPosTagSeq: ['CC', 'NNS', 'NN', 'VBZ', 'RB', 'IN', 'NFP', 'NFP', ',', 'CC', 'PRP', 'VBP', 'PRP', 'VBZ', 'RB', 'JJ', 'NN', '.']
['CC', 'DT', 'NNS', 'NN', 'VBZ', 'RB', 'IN', 'NFP', 'NFP', ',', 'CC', 'PRP', 'VBP', 'PRP', 





 55%|█████▍    | 17/31 [00:00<00:00, 160.22it/s]



 97%|█████████▋| 30/31 [00:00<00:00, 148.74it/s]



100%|██████████| 31/31 [00:00<00:00, 142.30it/s]



  0%|          | 0/109 [00:00<?, ?it/s]

...

reducedPosTagSeq: ['VB', 'NN', 'IN', 'NNP', 'IN', 'PRP$', 'NNS', 'VBN', 'RB']
['VB', 'DT', 'NN', 'IN', 'NNP', 'IN', 'PRP$', 'NNS', 'VBN', 'RB'] VB-DT-NN-IN-NNP-IN-PRP$-NNS-VBN-RB
A   , 
12 71 We aren’t sure what the length of the privilege variable is, so we write a script to look for changes in the admin page when we try different lengths, and plug that cookie into our web browser after the fact. A,I 70.0
reducedPosTagSeq: ['PRP', 'VBP', 'RB', 'JJ', 'WP', 'NN', 'IN', 'NN', 'NN', 'VBZ', ',', 'RB', 'PRP', 'VBP', 'NN', 'TO', 'VB', 'IN', 'NNS', 'IN', 'NN', 'NN', 'WRB', 'PRP', 'VBP', 'JJ', 'NNS', ',', 'CC', 'VB', 'NN', 'IN', 'PRP$', 'NN', 'NN', 'IN', 'NN', '.']
['PRP', 'VBP', 'RB', 'JJ', 'WP', 'DT', 'NN', 'IN', 'DT', 'NN', 'NN', 'VBZ', ',', 'RB', 'PRP', 'VBP', 'DT', 'NN', 'TO', 'VB', 'IN', 'NNS', 'IN', 'DT', 'NN', 'NN', 'WRB', 'PRP', 'VBP', 'JJ', 'NNS', ',', 'CC', 'VB', 'DT', 'NN', 'IN', 'PRP$', 'NN', 'NN', 'IN', 'DT', 'NN', '.'] PRP-VBP-RB-JJ-WP-DT-NN-IN-DT-NN-NN-VBZ-,-RB-PRP-VBP-DT-NN-TO





100%|██████████| 24/24 [00:00<00:00, 139.49it/s]



  0%|          | 0/36 [00:00<?, ?it/s]



 36%|███▌      | 13/36 [00:00<00:00, 125.53it/s]

reducedPosTagSeq: ['PRP', 'MD', 'RB', 'VB', 'NN', 'IN', 'JJ', 'NNS', 'CC', 'VB', 'JJ', 'NNS']
['PRP', 'MD', 'RB', 'VB', 'DT', 'NN', 'IN', 'JJ', 'NNS', 'CC', 'VB', 'DT', 'JJ', 'NNS'] PRP-MD-RB-VB-DT-NN-IN-JJ-NNS-CC-VB-DT-JJ-NNS
A   , 
9 19 But if you try to submit hitcon{ldgonaro}, you won't get any points despite submitting a working encryption key I 18.0
reducedPosTagSeq: ['CC', 'IN', 'PRP', 'VBP', 'TO', 'VB', 'NNP', '-RRB-', ',', 'PRP', 'MD', 'RB', 'VB', 'NNS', 'IN', 'VBG', 'VBG', 'NN', 'NN']
['CC', 'IN', 'PRP', 'VBP', 'TO', 'VB', 'NNP', '-RRB-', ',', 'PRP', 'MD', 'RB', 'VB', 'DT', 'NNS', 'IN', 'VBG', 'DT', 'VBG', 'NN', 'NN'] CC-IN-PRP-VBP-TO-VB-NNP--RRB--,-PRP-MD-RB-VB-DT-NNS-IN-VBG-DT-VBG-NN-NN
Rule: PRP-VB
I A  , A
9 20 The least-significant bit in each byte of a DES key is ignored during encryption/decryption I 18.0
reducedPosTagSeq: ['RBS', 'HYPH', 'JJ', 'NN', 'IN', 'NN', 'IN', 'NNP', 'NN', 'VBZ', 'VBN', 'IN', 'NN', 'SYM', 'NN']
['DT', 'RBS', 'HYPH', 'JJ', 'NN', 'IN', 'DT', 'NN'





 78%|███████▊  | 28/36 [00:00<00:00, 131.52it/s]



100%|██████████| 36/36 [00:00<00:00, 133.46it/s]

reducedPosTagSeq: ['PRP', 'VBP', 'IN', 'NN', 'CD', ',', 'EX', 'VBZ', 'JJ', 'NNP', 'NN', 'IN', 'NNP', 'NNP', 'IN', 'NNP']
['PRP', 'VBP', 'IN', 'NN', 'CD', ',', 'EX', 'VBZ', 'DT', 'JJ', 'NNP', 'NN', 'IN', 'NNP', 'NNP', 'IN', 'NNP'] PRP-VBP-IN-NN-CD-,-EX-VBZ-DT-JJ-NNP-NN-IN-NNP-NNP-IN-NNP
Rule: PRP-VB
Rule: EX-VBZ
I A I , A/I
8 19 Please refer to page 44. I 17.0
reducedPosTagSeq: ['UH', 'VB', 'IN', 'NN', 'CD', '.']
['UH', 'VB', 'IN', 'NN', 'CD', '.'] UH-VB-IN-NN-CD-.
I   , 
8 20 Actually we can do some CSRF injection here using \uff0d\uff0a A,I 17.0
reducedPosTagSeq: ['RB', 'PRP', 'MD', 'VB', 'NN', 'NN', 'RB', 'VBG', 'NNS']
['RB', 'PRP', 'MD', 'VB', 'DT', 'NN', 'NN', 'RB', 'VBG', 'NNS'] RB-PRP-MD-VB-DT-NN-NN-RB-VBG-NNS
Rule: PRP-MD-VB
A/I A  , A
8 21 Another interesting ariticle of CSRF injection I found is request splitting by Ryan Kelly I 20.0
reducedPosTagSeq: ['JJ', 'NN', 'IN', 'NNP', 'NN', 'PRP', 'VBD', 'VBZ', 'NN', 'NN', 'IN', 'NNP', 'NNP']
['DT', 'JJ', 'NN', 'IN', 'NNP', 'NN', 'PRP

In [57]:
# actionRules = ['PRP-MD-VB','PRP-RB-VB','PRP-VB',
#                 'PRP-MD-VBG','PRP-RB-VBG','PRP-VBG',
#                 'PRP-MD-VBN','PRP-RB-VBN','PRP-VBN',
#                 'PRP-MD-VBP','PRP-RB-VBP','PRP-VBP',
#                'PRP-MD-RB-VB'
#                 #'PRP-MD-VBZ','PRP-RB-VBZ','PRP-VBZ',
#                 'PRP-MD-VBD','PRP-RB-VBD','PRP-VBD']
actionRules = ["PRP-MD-VB","PRP-RB-VB","PRP-VB"]
# infoRules = ['EX-VBD','EX-VBP','EX-VBZ','VBD-IN','PRP-VBZ',
#             'NN-VBD-VBN','NNS-VBD-VBN','NN-VBD-VBG','NNS-VBD-VBG',
#             'NN-VBP-VBN','NNS-VBP-VBN','NN-VBP-VBG','NNS-VBP-VBG',
#             'NN-VBZ-VBN','NNS-VBZ-VBN',
#             'NN-MD-VB','NN-MD-RB-VB']

infoRules = ["NN-VB-NN","NN-VB-VB"]
filterPos = 'PDT|DT|IN|,|:|\.|JJ|SYM|TO'
output = []
#print (allData[1])
for eachFile in allData:
    for docid,lineno,content,enttype,edges in tqdm(eachFile):
        print (docid,lineno,content,enttype,edges)
        #input()
        if enttype.strip()=="-":
            continue
        posTag = []
        for word in nlp(content):
            posTag.append(word.tag_)
        posTagSeq = "-".join(posTag)
        # Removing Adverbs(RB%, RP%), Adjective(PDT,JJ%), Conjunction &support preposition(IN), Determiner(DT), Punctuation (,:.)
        
        reducedPosTagSeq = [x for x in posTag if not re.search(filterPos,x) ]
        reducedPosTagSeqMod = []
        for x in reducedPosTagSeq:
            if re.match("NN*",x):
                reducedPosTagSeqMod.append("NN")
            elif re.match("VB*",x):
                reducedPosTagSeqMod.append("VB")
            else:
                reducedPosTagSeqMod.append(x)
                
        print (reducedPosTagSeq)
        print (reducedPosTagSeqMod)
        posTagSeqNew = "-".join(reducedPosTagSeqMod)

        print ("reducedPosTagSeq:",reducedPosTagSeq)
        print (posTag,posTagSeq)
        predEnttypeAction = ''
        for eachRule in actionRules:
            if eachRule in posTagSeqNew:
                predEnttypeAction = 'A'
                print ("Rule:",eachRule)
                break
                
        predEnttypeInfo = ''
        for eachRule in infoRules:
            if eachRule in posTagSeqNew:
                predEnttypeInfo = 'I'
                print ("Rule:",eachRule)
                break

        if enttype in ["A,I","I,A","M"]:
            enttype = "A/I"
        pred=""
        if predEnttypeAction and predEnttypeInfo:
            pred = predEnttypeAction+"/"+predEnttypeInfo
        else:
            pred = predEnttypeAction+predEnttypeInfo
            
        output.append([enttype,pred])
        print (enttype, predEnttypeAction,predEnttypeInfo,",",pred)
        #input()
nodeAccuracy(output)









  0%|          | 0/34 [00:00<?, ?it/s]







 35%|███▌      | 12/34 [00:00<00:00, 110.04it/s]

6 1 We are given a nodejs API which we can read/write the notes A/I -
['PRP', 'VBP', 'VBN', 'NNS', 'NN', 'PRP', 'MD', 'VB', 'VB', 'NNS']
['PRP', 'VB', 'VB', 'NN', 'NN', 'PRP', 'MD', 'VB', 'VB', 'NN']
reducedPosTagSeq: ['PRP', 'VBP', 'VBN', 'NNS', 'NN', 'PRP', 'MD', 'VB', 'VB', 'NNS']
['PRP', 'VBP', 'VBN', 'DT', 'NNS', 'NN', 'WDT', 'PRP', 'MD', 'VB', 'SYM', 'VB', 'DT', 'NNS'] PRP-VBP-VBN-DT-NNS-NN-WDT-PRP-MD-VB-SYM-VB-DT-NNS
Rule: PRP-MD-VB
A/I A  , A
6 2 Follow the instruction on the landing page and visit /help, and we can see a hidden file fetcher API: A -
['VB', 'NN', 'NN', 'NN', 'CC', 'VB', 'NNP', 'CC', 'PRP', 'MD', 'VB', 'VBN', 'NN', 'NN', 'NN']
['VB', 'NN', 'NN', 'NN', 'CC', 'VB', 'NN', 'CC', 'PRP', 'MD', 'VB', 'VB', 'NN', 'NN', 'NN']
reducedPosTagSeq: ['VB', 'NN', 'NN', 'NN', 'CC', 'VB', 'NNP', 'CC', 'PRP', 'MD', 'VB', 'VBN', 'NN', 'NN', 'NN']
['VB', 'DT', 'NN', 'IN', 'DT', 'NN', 'NN', 'CC', 'VB', 'NNP', ',', 'CC', 'PRP', 'MD', 'VB', 'DT', 'VBN', 'NN', 'NN', 'NN', ':'] VB-DT-NN-









 82%|████████▏ | 28/34 [00:00<00:00, 120.58it/s]







100%|██████████| 34/34 [00:00<00:00, 127.72it/s]







  0%|          | 0/17 [00:00<?, ?it/s]







 76%|███████▋  | 13/17 [00:00<00:00, 127.64it/s]









['RB', 'VB', 'NN']
['RB', 'VB', 'NN']
reducedPosTagSeq: ['RB', 'VB', 'NN']
['RB', 'VB', 'DT', 'NN', '.'] RB-VB-DT-NN-.
A   , 
6 28 It's silly I didn't solve this one, because I forget to try the empty string in the path parameter..... A/I 28
['PRP', 'VBZ', 'PRP', 'VBD', 'RB', 'VB', 'NN', 'PRP', 'VBP', 'VB', 'NN', 'NN', 'NN']
['PRP', 'VB', 'PRP', 'VB', 'RB', 'VB', 'NN', 'PRP', 'VB', 'VB', 'NN', 'NN', 'NN']
reducedPosTagSeq: ['PRP', 'VBZ', 'PRP', 'VBD', 'RB', 'VB', 'NN', 'PRP', 'VBP', 'VB', 'NN', 'NN', 'NN']
['PRP', 'VBZ', 'JJ', 'PRP', 'VBD', 'RB', 'VB', 'DT', 'NN', ',', 'IN', 'PRP', 'VBP', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN', 'NN', '.'] PRP-VBZ-JJ-PRP-VBD-RB-VB-DT-NN-,-IN-PRP-VBP-TO-VB-DT-JJ-NN-IN-DT-NN-NN-.
Rule: PRP-VB
A/I A  , A
6 29 so I assume it must be /. A -
['CC', 'PRP', 'VBP', 'PRP', 'MD', 'VB']
['CC', 'PRP', 'VB', 'PRP', 'MD', 'VB']
reducedPosTagSeq: ['CC', 'PRP', 'VBP', 'PRP', 'MD', 'VB']
['CC', 'PRP', 'VBP', 'PRP', 'MD', 'VB', '.'] CC-PRP-VBP-PRP-MD-VB-.
Rule: PR

100%|██████████| 17/17 [00:00<00:00, 124.38it/s]







  0%|          | 0/31 [00:00<?, ?it/s]







 55%|█████▍    | 17/31 [00:00<00:00, 165.29it/s]

...

['NN', 'VB', 'NN', 'CC', 'VB', 'PRP', 'NNP', 'NN']
['NN', 'VB', 'NN', 'CC', 'VB', 'PRP', 'NN', 'NN']
reducedPosTagSeq: ['NN', 'VB', 'NN', 'CC', 'VB', 'PRP', 'NNP', 'NN']
['NN', 'VB', 'DT', 'JJ', 'NN', ',', 'CC', 'VB', 'PRP', 'IN', 'DT', 'NNP', 'NN', '.'] NN-VB-DT-JJ-NN-,-CC-VB-PRP-IN-DT-NNP-NN-.
Rule: NN-VB-NN
A  I , I
12 75 The extra data and hash values are read from the piped output of HashPump, all we need to do is put everything together in the right order. A,I 74.0
['NNS', 'CC', 'NN', 'NNS', 'VBP', 'VBN', 'NN', 'NNP', 'PRP', 'VBP', 'VB', 'VBZ', 'VB', 'NN', 'RB', 'NN']
['NN', 'CC', 'NN', 'NN', 'VB', 'VB', 'NN', 'NN', 'PRP', 'VB', 'VB', 'VB', 'VB', 'NN', 'RB', 'NN']
reducedPosTagSeq: ['NNS', 'CC', 'NN', 'NNS', 'VBP', 'VBN', 'NN', 'NNP', 'PRP', 'VBP', 'VB', 'VBZ', 'VB', 'NN', 'RB', 'NN']
['DT', 'JJ', 'NNS', 'CC', 'NN', 'NNS', 'VBP', 'VBN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NNP', ',', 'DT', 'PRP', 'VBP', 'TO', 'VB', 'VBZ', 'VB', 'NN', 'RB', 'IN', 'DT', 'JJ', 'NN', '.'] DT-JJ-NNS-CC-NN-N









100%|██████████| 24/24 [00:00<00:00, 137.40it/s]







  0%|          | 0/36 [00:00<?, ?it/s]







 33%|███▎      | 12/36 [00:00<00:00, 115.66it/s]

['PRP', 'VBP', 'NNS', 'VBP', 'RB', 'VBN', 'RP', 'NNS', 'RB', 'EX', 'VBP', 'CD', 'NNS']
['PRP', 'VB', 'NN', 'VB', 'RB', 'VB', 'RP', 'NN', 'RB', 'EX', 'VB', 'CD', 'NN']
reducedPosTagSeq: ['PRP', 'VBP', 'NNS', 'VBP', 'RB', 'VBN', 'RP', 'NNS', 'RB', 'EX', 'VBP', 'CD', 'NNS']
['IN', 'PRP', 'VBP', 'DT', 'NNS', 'WDT', 'VBP', 'RB', 'VBN', 'RP', 'IN', 'NNS', 'RB', ',', 'EX', 'VBP', 'CD', 'JJ', 'NNS'] IN-PRP-VBP-DT-NNS-WDT-VBP-RB-VBN-RP-IN-NNS-RB-,-EX-VBP-CD-JJ-NNS
Rule: PRP-VB
A/I A  , A
9 23 That's a small enough set that you can just submit them all until you get it right, and for this challenge, the accepted flag was hitcon{megnnaro} A,I 22.0
['VBZ', 'RB', 'VBN', 'PRP', 'MD', 'RB', 'VB', 'PRP', 'PRP', 'VBP', 'PRP', 'CC', 'NN', 'VBN', 'NN', 'VBD', 'NNS', '-RRB-']
['VB', 'RB', 'VB', 'PRP', 'MD', 'RB', 'VB', 'PRP', 'PRP', 'VB', 'PRP', 'CC', 'NN', 'VB', 'NN', 'VB', 'NN', '-RRB-']
reducedPosTagSeq: ['VBZ', 'RB', 'VBN', 'PRP', 'MD', 'RB', 'VB', 'PRP', 'PRP', 'VBP', 'PRP', 'CC', 'NN', 'VBN', 'NN', 









 72%|███████▏  | 26/36 [00:00<00:00, 121.66it/s]







100%|██████████| 36/36 [00:00<00:00, 128.01it/s]

['NN', 'NNP', 'NN', 'PRP', 'VBD', 'VBZ', 'NN', 'NN', 'NNP', 'NNP']
['NN', 'NN', 'NN', 'PRP', 'VB', 'VB', 'NN', 'NN', 'NN', 'NN']
reducedPosTagSeq: ['NN', 'NNP', 'NN', 'PRP', 'VBD', 'VBZ', 'NN', 'NN', 'NNP', 'NNP']
['DT', 'JJ', 'NN', 'IN', 'NNP', 'NN', 'PRP', 'VBD', 'VBZ', 'NN', 'NN', 'IN', 'NNP', 'NNP'] DT-JJ-NN-IN-NNP-NN-PRP-VBD-VBZ-NN-NN-IN-NNP-NNP
Rule: PRP-VB
I A  , A
8 22 We can inject two CRLFs to make the nodejs socket send another request! I 20.0
['PRP', 'MD', 'VB', 'CD', 'NNS', 'VB', 'NNS', 'NN', 'VB', 'NN']
['PRP', 'MD', 'VB', 'CD', 'NN', 'VB', 'NN', 'NN', 'VB', 'NN']
reducedPosTagSeq: ['PRP', 'MD', 'VB', 'CD', 'NNS', 'VB', 'NNS', 'NN', 'VB', 'NN']
['PRP', 'MD', 'VB', 'CD', 'NNS', 'TO', 'VB', 'DT', 'NNS', 'NN', 'VB', 'DT', 'NN', '.'] PRP-MD-VB-CD-NNS-TO-VB-DT-NNS-NN-VB-DT-NN-.
Rule: PRP-MD-VB
Rule: NN-VB-NN
I A I , A/I
8 23 The rest is trivial - -
8 24 Just send the request to /flag/:token A 20.0
['RB', 'VB', 'NN', 'ADD']
['RB', 'VB', 'NN', 'ADD']
reducedPosTagSeq: ['RB', 'VB